# Connect with Google Drive for Google Colab

In [1]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import os 

print(os.getcwd())
print(os.listdir())

os.chdir('drive/My Drive/Projects/Dog Breed 2.0/')

/content
['.config', 'drive', 'sample_data']


# Import required libraries

In [3]:
import numpy as np
from keras import regularizers
import tensorflow.keras as keras
from keras import backend as K
from keras.models import Model, Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, Dropout
from keras.models import Sequential
from keras.layers import Activation, Lambda, GlobalAveragePooling2D
from keras.layers.core import Dense, Flatten, Dropout
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.applications.inception_v3 import InceptionV3
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from collections import Counter

Using TensorFlow backend.


# Preprocessing

In [8]:
train_path = 'dataset/without_downsampled_data/train/'
cls_ = os.listdir(train_path)
no_cls = len(os.listdir(train_path))

img_height = 229
img_width = 229
batch_size = 128
epoch = 100
model_name = "inceptionV3_KaggelDataset_WithoutDownsampled_Droupout0_LrNONE_.h5"


train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   validation_split=0.1)


train_batches = train_datagen.flow_from_directory(
                                                train_path, 
                                                target_size=(img_height, img_width),
                                                classes=cls_, 
                                                batch_size=batch_size,
                                                subset='training')


validation_batches = train_datagen.flow_from_directory(
                                                    train_path,
                                                    target_size=(img_height, img_width),
                                                    classes=cls_,
                                                    batch_size=batch_size,
                                                    subset='validation')


counter = Counter(train_batches.classes)
max_value = float(max(counter.values()))


CLASS_WEIGHTS = {classid: max_value / num_occurences
                 for classid, num_occurences in counter.items()}

Found 7451 images belonging to 120 classes.
Found 770 images belonging to 120 classes.


# Transfer Learning

In [0]:
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

base_model.trainable = False

model = Sequential()

model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(1024, activation='relu'))


# enable if required
# model.add(Dense(1024, activation='relu', kernel_regularizer = regularizers.l2(0.0001), 
#                                     activity_regularizer = regularizers.l1(0.0001)))
#model.add(Dropout(0.20))
model.add(Dense(no_cls, activation='softmax'))

print(model.summary())

W0730 14:04:03.281349 139779522197376 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0730 14:04:03.343427 139779522197376 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0730 14:04:03.353987 139779522197376 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0730 14:04:03.392703 139779522197376 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0730 14:04:03.394120 1397795221

87916544/87910968 [==============================] - 4s 0us/step


In [0]:
model.compile(Adam(lr=0.01), loss='categorical_crossentropy', metrics=['accuracy'])

print("Model Compiled Successfully")

modelcheckpoint = ModelCheckpoint('Outputs/keras/'+ model_name,
                                  monitor='val_acc',
                                  mode='max',
                                  verbose=1,
                                  save_best_only=True)


lr_callback = ReduceLROnPlateau(min_lr=0.000001)


callback_list = [modelcheckpoint, lr_callback]


history = model.fit_generator(
    train_batches, steps_per_epoch = train_batches.samples // batch_size,
    validation_data=validation_batches, validation_steps = validation_batches.samples // batch_size,
    epochs=epoch, verbose=1, 
    callbacks = callback_list,
    class_weight = CLASS_WEIGHTS,
)

# Graphs

In [0]:
from keras.callbacks import History

print(history.history.keys())

# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()